In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import collections
import sklearn.model_selection

from classes import *
from files import create_outfile_each_fold, create_outfile_mean, get_path_outfile_each_fold, get_path_mean, save_figs
from gpu import set_avx_avx2, set_gpu
from metrics import calculate_iou_dice, dice_loss, jaccard_distance_loss, jaccard_distance, dice_coef
from model import cfg_model, train_model

In [ ]:
tensorflow.__version__

# GPU

In [ ]:
set_avx_avx2()
set_gpu()

In [ ]:
cfg = {
    "channel": 1,
    "batch_size": 4,
    "cross_validation": 5,
    "epochs": 75,
    "image_size": 400,
    "learning_rate": 0.001,
    "random_state": 1234,
    "test_size": 0.2,
    "val_size": 0.05,
    "path_base": "dataset",
    "path_out": "out"
}
path_images = os.path.join(cfg["path_base"], "grayscale/images_grayscale_400_400")
path_mask = os.path.join(cfg["path_base"], "mask")

In [ ]:
piperaceae_dataset = Dataset(cfg, path_images, path_mask)
list_images = getattr(piperaceae_dataset, "list_images")
len(getattr(piperaceae_dataset, "list_images"))

In [ ]:
x = numpy.array(list([l for l in list_images]), dtype=object)
y = numpy.array(list([getattr(l["image"], "label").value for l in list_images]))

collections.Counter(y.tolist())

In [ ]:
index_train_test = sklearn.model_selection.StratifiedKFold(n_splits=cfg["cross_validation"], shuffle=True, random_state=cfg["random_state"])
list_index = list([Index(fold, index_train, index_test) for fold, (index_train, index_test) in enumerate(index_train_test.split(x, y))])
len(list_index)

In [ ]:
import itertools
list_index_test = list([getattr(i, "index_test").tolist() for i in list_index])
list_all_index_test = list(itertools.chain(*list_index_test))
print(collections.Counter(sorted(list_all_index_test)), sep="")

In [ ]:
set(numpy.arange(0, 375).tolist()) ^ set(list_all_index_test)

In [ ]:
def get_image(cfg, list_images):
    images = list([getattr(image["image"], "image") for image in list_images])
    mask = list([getattr(image["mask"], "image") for image in list_images])
    x = numpy.array(images).reshape((len(images), cfg["image_size"], cfg["image_size"], cfg["channel"]))
    y = numpy.array(mask).reshape((len(mask), cfg["image_size"], cfg["image_size"], 1))
    return x, y


list_result = list([])
list_elapsed_time = list([])
path = get_path_mean(cfg)
pathlib.Path(path).mkdir(parents=True, exist_ok=True)
b = list([])
for index in list_index:
    fold = str(getattr(index, "fold"))
    print(fold)
    path_outfile_each_fold = get_path_outfile_each_fold(fold, path)
    pathlib.Path(path_outfile_each_fold).mkdir(parents=True, exist_ok=True)

    index_train = getattr(index, "index_train")
    index_test = getattr(index, "index_test")

    x_train_o, y_train_o = x[index_train], y[index_train]
    x_test_o, y_test_o = x[index_test], y[index_test]
    x_train_o, x_val_o, y_train_o, y_val_o = sklearn.model_selection.train_test_split(x_train_o, y_train_o, test_size=cfg["val_size"], random_state=cfg["random_state"])

    x_train, y_train = get_image(cfg, x_train_o)
    x_test, y_test = get_image(cfg, x_test_o)
    x_val, y_val = get_image(cfg, x_val_o)

    print(x_train.shape, y_train.shape)
    print(x_test.shape, y_test.shape)
    print(x_val.shape, y_val.shape)

    checkpointer, reduce_learning_rate, steps_per_epoch, strategy, train_generator, unet_filename = cfg_model(cfg, fold, path_outfile_each_fold, x_train, y_train)

    if os.path.exists(unet_filename):
        model = tensorflow.keras.models.load_model(unet_filename, custom_objects = {"jaccard_distance_loss": jaccard_distance_loss, "dice_coef": dice_coef, "jaccard_distance": jaccard_distance })
    else:
        elapsed_time, model = train_model(cfg, checkpointer, fold, path_outfile_each_fold, reduce_learning_rate, steps_per_epoch, strategy, train_generator, x_val, y_val)

    metrics = calculate_iou_dice(model, x_test, x_train, x_val, y_test, y_train, y_val)
    list_elapsed_time.append(elapsed_time)
    list_result.append(metrics)
    create_outfile_each_fold(elapsed_time, fold, metrics, path_outfile_each_fold)
    save_figs(cfg, model, path_outfile_each_fold, x_test_o, x_train_o, x_val_o, y_test_o, y_train_o, y_val_o)
    break

In [ ]:
create_outfile_mean(cfg, list_elapsed_time, list_result, path)
len(list_result)